# EDA con Enfoque en CAC y LTV

Este cuaderno se centra en **preguntas de negocio** que generan valor al entender la relación entre:
- **CAC**: Costo de Adquisición de Cliente.
- **LTV**: Valor de Vida del Cliente.

Adicionalmente, explora aspectos clave como **suscripción**, **descuentos**, **categoría** del producto, **frecuencia de compras**, **método de pago**, etc. Con estas preguntas podrás tomar **decisiones estratégicas** sobre tu modelo de negocio, campañas de marketing y retención de clientes.

## Contenido:
1. **Importar librerías y cargar datos**
2. **Inspección inicial** (dimensiones, tipos de datos, nulos)
3. **Limpieza básica** (si es necesaria)
4. **Creación de columna** ratio `LTV/CAC` (rentabilidad por cliente)
5. **Preguntas de negocio** (groupby, filtros, etc.)
6. **Conclusiones**

In [4]:
!wget !wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv

--2025-03-07 18:32:44--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-03-07 18:32:44--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv [following]
--2025-03-07 18:32:44--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HT

## 1. Importar librerías y cargar datos
Asegúrate de tener tu archivo CSV con las columnas mencionadas: `CAC`, `LTV`, `Purchase Amount (USD)`, etc. Ajusta la ruta según tu archivo real.

In [5]:
import pandas as pd
import numpy as np


# Cargar el dataset
df = pd.read_csv("shopping_trends_LTV_CAC.csv")


# Mostramos las primeras filas para verificar
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Next Day Air,Yes,Yes,49,PayPal,Weekly,NaN,1914,26.957746,26.957746
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574


## 2. Inspección inicial
1. **Dimensiones (filas x columnas)**
2. **Info general**: tipos de dato, nulos, etc.
3. **Valores nulos** por columna.

Si existen valores nulos en columnas importantes (por ejemplo, `CAC`, `LTV`, `Customer ID`), evalúa si los eliminarás o imputarás.

In [6]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase Amount (USD)     3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review Rating             3900 non-null   float64
 11  Subscription Status       3900 non-null   object 
 12  Payment Method            3900 non-null   object 
 13  Shipping Type             3900 non-null   object 
 14  Discount

## 3. Limpieza básica
Por ejemplo, si detectas `CAC <= 0` o `LTV <= 0` y consideras que no tienen sentido en tu negocio, podrías filtrar esos registros. Ajusta según tu criterio.

In [7]:
df[df['CAC'] <= 0]
df[df['LTV'] <= 0]

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio


## 4. Creación de columna: Ratio LTV/CAC
Es una métrica clave de la **rentabilidad** del cliente. Un valor > 1 indica que, en teoría, el cliente retorna más de lo que cuesta adquirirlo.

In [8]:
RatioLTVCAC = df['LTV'] / df['CAC']
df['Ratio LTV/CAC'] = RatioLTVCAC
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio,Ratio LTV/CAC
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Yes,Yes,49,PayPal,Weekly,NaN,1914,26.957746,26.957746,NaN
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574,27.659574


## 5. Preguntas de Negocio
En vez de usar gráficas o correlaciones, nos centramos en **preguntas que generen valor** al análisis. Cada una se resuelve con **groupby, filtros o estadísticas** simples.

1. **¿Cuántos clientes únicos hay y cuál es la edad promedio?**
2. **¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?**
3. **¿Cuál es la ratio LTV/CAC promedio?**
4. **¿Qué categoría tiene el mayor LTV en promedio?**
5. **¿En cuál categoría se justifica más el CAC?** (categoría con mayor ratio LTV/CAC)
6. **¿Los clientes suscritos tienen LTV mayor que los no suscritos?**
7. **¿Cuál es el método de pago más frecuente y su LTV promedio?**
8. **¿Quiénes usan descuentos con mayor frecuencia?** (Por ejemplo, ¿cierto género, cierto rango de edad?)
9. **¿Cuántos clientes aprovechan códigos promocionales (Promo Code Used)?** y su ratio LTV/CAC
10. **¿Cuál es la frecuencia de compra más común (Frequency of Purchases)?** y su LTV asociado.
11. **¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?**
12. **Entre los clientes con `Previous Purchases` > X, cuál es el gasto promedio y LTV promedio?**


### 5.1 ¿Cuántos clientes únicos hay y cuál es la edad promedio?

In [9]:
UniqueCustomers = df['Customer ID'].nunique()
AverageAge = df['Age'].mean()
print(f"Número de clientes únicos: {UniqueCustomers}")
print(f"Edad promedio de los clientes: {AverageAge:.2f} años")

Número de clientes únicos: 3900
Edad promedio de los clientes: 44.07 años


### 5.2 ¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?

In [10]:
TotalCAC = df['CAC'].mean()
TotalLTV = df['LTV'].mean()
print(f"CAC promedio: {TotalCAC:.2f}")
print(f"LTV promedio: {TotalLTV:.2f}")

CAC promedio: 66.29
LTV promedio: 1108.94


### 5.3 ¿Cuál es la ratio LTV/CAC promedio?

In [11]:
TotalRatioLTVCAC = df['Ratio LTV/CAC'].mean()
print(TotalRatioLTVCAC)

22.446982949947316


### 5.4 ¿Qué categoría tiene el mayor LTV en promedio?

In [12]:
TopCategories = df.groupby('Category')['LTV'].mean().sort_values(ascending=False)
print(TopCategories)

Category
Accessories    1145.992742
Footwear       1099.308848
Clothing       1095.688543
Outerwear      1056.040123
Name: LTV, dtype: float64


### 5.5 ¿En cuál categoría se justifica más el CAC? (mayor ratio LTV/CAC)

In [13]:
JustifiedCAC = df.groupby('Category')['Ratio LTV/CAC'].mean().sort_values(ascending=False)
print(JustifiedCAC)

Category
Accessories    29.281313
Outerwear      26.319371
Clothing       16.845818
Footwear             NaN
Name: Ratio LTV/CAC, dtype: float64


### 5.6 ¿Los clientes suscritos tienen LTV mayor que los no suscritos?

In [14]:
SuscribersCheck = df.groupby('Subscription Status')['LTV'].mean().sort_values(ascending=False)
print(SuscribersCheck)

Subscription Status
No     1113.425009
Yes    1096.831909
Name: LTV, dtype: float64


### 5.7 ¿Cuál es el método de pago más frecuente y su LTV promedio?

In [15]:
PaymentMethodMean = df.groupby('Payment Method')['LTV'].mean().sort_values(ascending=False)
print (PaymentMethodMean)
MostUsed = df['Payment Method'].value_counts()
MostUsed

Payment Method
Cash             1151.208333
Bank Transfer    1139.090190
Credit Card      1097.510057
Venmo            1090.260337
Debit Card       1089.439179
PayPal           1087.108150
Name: LTV, dtype: float64


,count
Payment Method,
Credit Card,696
Venmo,653
Cash,648
PayPal,638
Debit Card,633
Bank Transfer,632


5.8 ¿Quiénes usan descuentos con mayor frecuencia?
Podemos ver si hay algún grupo (por género, rango de edad, etc.) que destaca. A modo de ejemplo, segmentaremos por género:

In [ ]:
discount_users = df[df['Discount Applied'] == 'Yes']
discount_by_gender = discount_users.groupby('Gender')['Customer ID'].count()
discount_by_gender

,Customer ID
Gender,
Male,1677


Podrías repetir el ejercicio segmentando por **Age** (definiendo rangos) o por **Location**.

### 5.9 ¿Cuántos clientes aprovechan códigos promocionales y cuál es su ratio LTV/CAC?
El uso de promos puede modificar la rentabilidad.

In [35]:
PromoUsed = df[df['Promo Code Used'] == 'Yes']
PromoUsedCount = PromoUsed.groupby('Promo Code Used')['Ratio LTV/CAC'].count()
print(PromoUsedCount)
PromoUsed = df[df['Promo Code Used'] == 'Yes']
PromoUsedRatio = PromoUsed.groupby('Promo Code Used')['Ratio LTV/CAC'].mean()
PromoUsedRatio

Promo Code Used
Yes    1418
Name: Ratio LTV/CAC, dtype: int64


,Ratio LTV/CAC
Promo Code Used,
Yes,22.705036


### 5.10 ¿Cuál es la frecuencia de compra más común (Frequency of Purchases) y su LTV?
Esto ayuda a ver si, por ejemplo, los clientes que compran semanalmente tienen un LTV más alto.

In [36]:
GenderCheck = df['Frequency of Purchases'].value_counts()
print(GenderCheck)
LTV = df.groupby('Frequency of Purchases')['LTV'].mean().sort_values(ascending=False)
LTV.round(2)

Frequency of Purchases
Every 3 Months    584
Annually          572
Quarterly         563
Monthly           553
Bi-Weekly         547
Fortnightly       542
Weekly            539
Name: count, dtype: int64


,LTV
Frequency of Purchases,
Bi-Weekly,1182.28
Weekly,1111.23
Annually,1107.47
Fortnightly,1106.50
Quarterly,1098.23
Monthly,1087.00
Every 3 Months,1072.97


### 5.11 ¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?

In [26]:
TopLocation = df.groupby('Location')['Ratio LTV/CAC'].mean().sort_values(ascending=False).head(1)
TopLocation

,Ratio LTV/CAC
Location,
New York,32.785849


### 5.12 Entre los clientes con `Previous Purchases` > X, ¿cuál es el gasto promedio y el LTV promedio?
Define un umbral X según tu contexto. Por ejemplo, X = 20:

In [29]:
PrevPurchases = df[df['Previous Purchases'] > 20]
AveragePurchaseAmount = PrevPurchases['Purchase Amount (USD)'].mean()
AverageLTV = PrevPurchases['LTV'].mean()
print(f"Gasto promedio: {AveragePurchaseAmount:.2f} USD")
print(f"LTV promedio: {AverageLTV:.2f}")

Gasto promedio: 59.70 USD
LTV promedio: 1114.30


## 6. Conclusiones

En este Notebook, hemos abordado un **EDA orientado a negocio** en el que:
1. **Identificamos** la base total de clientes y su edad promedio.
2. **Calculamos** el CAC y LTV promedio, y la ratio LTV/CAC global.
3. **Segregamos** por categorías, suscripción, uso de descuentos, métodos de pago, etc.
4. **Resolvimos** preguntas de negocio que ayudan a:
   - **Identificar** cuáles categorías o ubicaciones ofrecen mejor rentabilidad (ratio LTV/CAC).
   - **Entender** si la suscripción eleva el LTV.
   - **Explorar** la efectividad de promos y descuentos.
   - **Diferenciar** frecuencias de compra y ver su impacto en LTV.
   - **Focalizar** inversiones de marketing en los segmentos más prometedores.

Este tipo de **análisis** proporciona las bases para **estrategias** de retención, adquisición y segmentación de clientes. ¡Espero que te sea útil para **generar valor** en tu negocio!